In [34]:
import pandas as pd
import pickle

import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.linear_model import  Ridge, Lasso
from sklearn.preprocessing import StandardScaler

%config InlineBackend.figure_format = 'svg'
# this statement allows the visuals to render within your Jupyter Notebook
%matplotlib inline 

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 25)
pd.set_option('display.precision', 3)

import seaborn as sns

In [2]:
with open('data/pickles/full_feature_log_inflected.pickle', 'rb') as to_read:
    df = pickle.load(to_read)

In [4]:
X = df.drop('log_grad_rate', axis = 1)
y = df.log_grad_rate

## Lasso

In [5]:
kf = KFold(n_splits = 5, shuffle = True, random_state = 42)

In [6]:
k_splits = kf.split(X,y)

In [7]:
lasso = Lasso(alpha = .009)

In [8]:
train_r2 = []
val_r2 = []
for train, val in k_splits:
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X.iloc[train])
    y_train = y.iloc[train]
    lasso.fit(X_train, y_train)
    train_r2.append(lasso.score(X_train, y_train))
    
    X_val = scaler.transform(X.iloc[val])
    y_val = y.iloc[val]
    val_r2.append(lasso.score(X_val, y_val))
    

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  # Remove the CWD from sys.path while we load stuff.
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with inpu

In [9]:
print(sum(train_r2)/len(train_r2))
print(sum(val_r2)/len(val_r2))

0.6485429414399766
0.5117785033196245


In [10]:
for feature, coef in zip(list(X), lasso.coef_):
    print(feature, coef)

Student_Count_Total -0.06854131159382504
Zip_Mean_Income -0.0
Fully Accessible -0.000923148480526953
Generally accessible 0.0
Student_Count_Asian_perc 0.0
Student_Count_Asian_Pacific_Islander_perc -0.08719708381382663
Student_Count_Black_perc 0.07089903375325464
Student_Count_English_Learners_perc 0.11191885079961585
Student_Count_Ethnicity_Not_Available_perc 0.050168674798737566
Student_Count_Hawaiian_Pacific_Islander_perc -0.019698395255543698
Student_Count_Hispanic_perc -0.2155447762746349
Student_Count_Low_Income_perc 0.0989251447265497
Student_Count_Multi_perc -0.21477039841360557
Student_Count_Native_American_perc 0.11939071645313
Student_Count_Other_Ethnicity_perc 0.0
Student_Count_Special_Ed_perc 0.25513965077597367
Student_Count_White_perc -0.0
Classroom_Languages_count -0.013164640516607106
Blue 0.03526704949045995
Brown -0.0
Green -0.07826264411360434
Orange 0.0
Pink 0.0
Purple 0.02596094478549125
Red -0.0036483053839929036
Dress_Code_dummie -0.04815576348112302
grades_offer

## Linear Model

In [25]:
X = X.drop(columns = ['Zip_Mean_Income', 'Generally accessible', 'Student_Count_Asian_perc', 
           'Student_Count_Asian_perc', 'Student_Count_Other_Ethnicity_perc','Student_Count_White_perc',
          'Brown', 'Orange', 'Pink', 'gender_f_bn'])

In [26]:
kf = KFold(n_splits = 5, shuffle=True, random_state=42)
kf_splits = kf.split(X,y)

r2_train = []
r2_val =   []
for train, val in kf_splits:
    lm = LinearRegression()
    X_train, y_train = X.iloc[train], y.iloc[train]
    X_val, y_val     = X.iloc[val], y.iloc[val]
    
    lm.fit(X_train, y_train)
    r2_train.append(lm.score(X_train, y_train))
    r2_val.append(lm.score(X_val, y_val))

In [27]:
print(sum(r2_train)/len(r2_train))

0.6535584479003873


In [28]:
print(sum(r2_val)/len(r2_train))

0.5171118622329536


In [29]:
## Train and score on whole set

In [30]:
lm = LinearRegression()
lm.fit(X,y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [31]:
lm.coef_

array([-1.15934650e-04, -5.87169544e-02, -2.00491593e+02,  2.13864023e-02,
        1.68592302e+00,  4.74507763e+00, -1.00072092e+01, -9.88270181e-01,
        4.76406414e-01, -2.45158225e+01,  3.95853001e+01,  1.72317549e+00,
       -1.50854912e-02,  1.44199860e-01, -2.45747570e-01,  6.86921903e-02,
        2.85799778e-03, -3.25615202e-02,  9.71968763e-02, -2.51776854e-01])

In [32]:
for feature, coef in zip(list(X),lm.coef_ ):
    print(feature, coef)

Student_Count_Total -0.00011593464954370333
Fully Accessible -0.058716954405733386
Student_Count_Asian_Pacific_Islander_perc -200.4915933278869
Student_Count_Black_perc 0.021386402274190596
Student_Count_English_Learners_perc 1.6859230233470734
Student_Count_Ethnicity_Not_Available_perc 4.7450776295227035
Student_Count_Hawaiian_Pacific_Islander_perc -10.007209194058493
Student_Count_Hispanic_perc -0.9882701812184495
Student_Count_Low_Income_perc 0.476406413966596
Student_Count_Multi_perc -24.51582249020286
Student_Count_Native_American_perc 39.58530008161357
Student_Count_Special_Ed_perc 1.723175486912006
Classroom_Languages_count -0.015085491191372856
Blue 0.1441998598989975
Green -0.24574756953950053
Purple 0.06869219034157005
Red 0.0028579977821253416
Dress_Code_dummie -0.03256152015289709
grades_offered_count 0.09719687633101778
charter -0.2517768540523275


In [39]:
df.head(15).sort_values('Student_Count_Black_perc', ascending=False)

,Student_Count_Total,Zip_Mean_Income,Fully Accessible,Generally accessible,Student_Count_Asian_perc,Student_Count_Asian_Pacific_Islander_perc,Student_Count_Black_perc,Student_Count_English_Learners_perc,Student_Count_Ethnicity_Not_Available_perc,Student_Count_Hawaiian_Pacific_Islander_perc,Student_Count_Hispanic_perc,Student_Count_Low_Income_perc,Student_Count_Multi_perc,Student_Count_Native_American_perc,Student_Count_Other_Ethnicity_perc,Student_Count_Special_Ed_perc,Student_Count_White_perc,Classroom_Languages_count,Blue,Brown,Green,Orange,Pink,Purple,Red,Dress_Code_dummie,grades_offered_count,charter,gender_f_bn,log_grad_rate
10,225,53353.0,0,1,0.000,0.0,0.978,0.000,0.000,0.000,0.013,0.876,0.009,0.000,0.0,0.262,0.000,1.0,1,0,0,0,0,0,0,1,4,1,0,3.581
11,176,57770.0,0,1,0.000,0.0,0.960,0.000,0.000,0.000,0.023,0.511,0.011,0.006,0.0,0.295,0.000,1.0,1,0,0,0,0,0,0,1,4,1,0,3.754
13,269,57770.0,1,0,0.007,0.0,0.807,0.015,0.000,0.000,0.160,0.677,0.004,0.000,0.0,0.257,0.022,1.0,1,0,0,0,1,0,1,1,4,0,1,3.135
5,184,57770.0,0,0,0.000,0.0,0.777,0.027,0.000,0.000,0.196,0.967,0.011,0.011,0.0,0.272,0.005,0.0,0,0,0,0,0,0,0,0,4,0,1,4.582
4,199,53353.0,0,0,0.000,0.0,0.774,0.010,0.000,0.000,0.181,0.874,0.000,0.015,0.0,0.291,0.030,0.0,0,0,0,0,0,0,0,0,4,0,1,4.588
12,245,53353.0,1,0,0.008,0.0,0.771,0.012,0.000,0.000,0.184,0.906,0.004,0.000,0.0,0.249,0.033,1.0,1,0,0,0,1,0,1,1,4,0,1,3.418
8,38,53353.0,0,0,0.000,0.0,0.737,0.053,0.000,0.000,0.211,0.974,0.000,0.000,0.0,0.158,0.053,1.0,0,0,0,0,1,0,0,0,7,0,1,4.364
9,34,57770.0,0,0,0.000,0.0,0.735,0.088,0.000,0.000,0.265,0.735,0.000,0.000,0.0,0.088,0.000,1.0,0,0,0,0,1,0,0,0,7,0,1,4.343
6,954,53353.0,0,1,0.034,0.0,0.309,0.084,0.002,0.002,0.635,0.853,0.002,0.003,0.0,0.148,0.013,2.0,1,0,0,0,1,0,0,1,4,1,1,2.815
7,931,57770.0,0,1,0.031,0.0,0.303,0.104,0.003,0.001,0.642,0.840,0.004,0.003,0.0,0.142,0.012,2.0,1,0,0,0,1,0,0,1,4,1,1,2.028
